# Kodeeksempler

Importerer funksjoner, og henter enheter fra Assetsystem

In [ ]:
. .\functions.ps1

$3SCredential = Import-Clixml .\3StepToken.cred

$3SauthToken = Get-3StepAuthToken -Endpoint $config."3StepTokenEndpoint" -Credential $3SCredential

$3SDevices = Get-3StepDevices -authToken $3SauthToken -Endpoint $config."3StepServiceEndpoint" | Where-Object { $_.serialNumber -eq "NXVRVED00K21216A227600" }  | Select-Object name,serialNumber,costcenter

$3SDevices

Koble til Azure AD og Intune for å finne ut hvem som bruker enheten

In [ ]:
$AADCredentials = Import-Clixml .\AADToken.cred

$authToken = Get-ApplicationAuthToken -Credential $AADCredentials

$IntuneDevice = Get-GraphData -uri "https://graph.microsoft.com/beta/deviceManagement/managedDevices?`$filter=serialNumber eq '$($3SDevices.serialnumber)'" -authToken $authToken | Select-Object id,serialNumber,deviceName,model,userPrincipalName

$IntuneDevice | Format-List

Henter informasjon om eleven teeljon@hortenskolen.no

In [ ]:
$elevInfo = Invoke-RestMethod -Uri "https://graph.microsoft.com/beta/users/$($IntuneDevice.userPrincipalName)?`$select=id,userPrincipalName,displayName,extension_fe2174665583431c953114ff7268b7b3_Education_SyncSource_SchoolId" -Headers $authToken | Select-Object id,displayName,userPrincipalName,extension_fe2174665583431c953114ff7268b7b3_Education_SyncSource_SchoolId

$elevInfo | Format-List


Finner skolen til teeljon@hortenskolen.no

In [ ]:
$skoleData = Get-GraphData -uri "https://graph.microsoft.com/beta/education/schools" -authToken $authToken | Where-Object { $_.externalId -eq $elevInfo.extension_fe2174665583431c953114ff7268b7b3_Education_SyncSource_SchoolId }

$skoleData

Flytter enheten til riktig skole i eksternt assetsystem

In [ ]:
Set-3StepDeviceCostCenter -Endpoint $config."3StepServiceEndpoint" -DeviceID $IntuneDevice.serialNumber -CostCenter $skoleData.displayName -authToken $3SauthToken